In [3]:
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten
import tensorflow_io as tfio
from matplotlib import pyplot as plt
import os

In [6]:
train_path_real = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/training/real/*.wav'
train_path_fake = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/training/fake/*.wav'

test_path_real = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/testing/real/'
test_path_fake = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/testing/fake/'

validation_path_real = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/validation/real/'
validation_path_fake = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/validation/fake/'

In [7]:
POS = train_path_real
NEG = train_path_fake

In [8]:
pos = tf.data.Dataset.list_files(POS)
neg = tf.data.Dataset.list_files(NEG)

positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))

In [9]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [14]:
def preprocess(file_path, label): 
    for i in os.listdir(file_path):
        i = file_path.decode() + "/" + i.decode()
        wav = load_wav_16k_mono(i)
        wav = wav[:48000]
        zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
        wav = tf.concat([zero_padding, wav],0)
        spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
        spectrogram = tf.abs(spectrogram)

        spectrogram = tf.expand_dims(spectrogram, axis=2)
        return spectrogram, label

filepath, label = positives.shuffle(buffer_size=10000).as_numpy_iterator().next()

spectrogram, label = preprocess(filepath, label)

NotADirectoryError: [Errno 20] Not a directory: b'/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/for-2seconds/training/real/file26358.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav'